In [ ]:
!pip install fosforml

## Sklearn Models Registration

### session creation

In [1]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

### modeling

#### Multiclass classification

In [ ]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

iris_dataset = load_iris() 


In [ ]:

feature_names = m_dataset.feature_names
u_features = [col.replace(" ","_").replace("(","").replace(")","") for col in feature_names]
m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df.columns = m_df.columns.str.upper()

m_df["Target"] = m_dataset.target


x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

In [ ]:
x_train

In [ ]:
from fosforml import register_model
register_model(
  model_obj=model,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="RandomMulticlassMOdelForTesting",
  dataset_source="Snowflake",
  name="Mahesh_sklearn_28Aug",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

In [ ]:
x_test_df = session.create_dataframe(x_test)

In [ ]:
from fosforml.model_manager import DatasetManager

dataset_manager = DatasetManager(model_name="MODEL_5AABC4FE_FC3D_4121_9E11_52C83C9FECC9_FDC_RANDOMMULTICLASSMODELFORTESTING", version_name="v2", session=session)
dataset_manager.upload_datasets(session=session, datasets={"x_train": x_train_df})

#### Custom Model using score_func

In [ ]:
from snowflake.ml.model import custom_model
import pandas as pd

In [ ]:
class CustomTestModel(custom_model.CustomModel):
    def __init__(self, context: custom_model.CustomModel) -> None:
        super().__init__(context)

    @custom_model.inference_api
    def predict(self, input_data: pd.DataFrame) -> pd.DataFrame:
        score_func = self.context.model_ref('feature_preproc')
        model = self.context.model_ref('model')
        return score_func(
            model,
            input_data
        )

In [ ]:
def feature_preproc(model,input_data):
    return pd.DataFrame(model.predict(input_data),columns=["Predicted"])

mc = custom_model.ModelContext(
    models={
        'model': model,
        'feature_preproc': feature_preproc
    }
)

In [ ]:
custom_test_model = CustomTestModel(mc)

In [ ]:
custom_model_predicted = custom_test_model.predict(x_test)

In [ ]:
custom_model_predicted.head()

In [ ]:
# custom_model_predicted

In [ ]:
from fosforml.model_manager import snowflakesession
session_instance = snowflakesession()
params = session_instance.connection_params

In [ ]:
params

In [3]:
from snowflake.ml.registry import Registry

In [4]:
# model_reg = Registry(
#     session=session,
#     database_name="FDC_DEV_VISHWASMAHESHWARI",
#     schema_name="FDC_DEV_SCHMEA"
# )

model_reg = Registry(
    session=session
)

In [5]:
m = model_reg.get_model('MODEL_8D3FB8A6_A886_4236_9B35_73EB1303C4BF_FDC_MAHESH_SKLEARN_28AUG')

In [6]:
mv = m.version('V1')

In [7]:
from snowflake.snowpark.session import Session
from snowflake.ml.registry.registry import Registry
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from snowflake.snowpark.functions import col, when
from snowflake.snowpark.types import StringType, DateType, BooleanType
import re

In [8]:
def append_prediction_cols(original_df, predict_df, cols):
    for col_name in cols:
        proba_col = predict_df.select(col(col_name)).to_pandas()
        original_df[col_name] = proba_col[col_name]
    return original_df

In [ ]:
original_data = session.table('Iris_inference_dataset')
remote_prediction = mv.run(original_data, function_name="predict")
original_data = original_data.to_pandas()
predict_columns = [col for col in remote_prediction.columns if col not in original_data.columns]
print(predict_columns)
# original_data = append_prediction_cols(original_data, remote_prediction, predict_columns)

In [15]:
proba_col = remote_prediction.select(col('"output_feature_0"')).to_pandas()

In [16]:
proba_col

,output_feature_0
0,1
1,0
2,1
3,1
4,0
5,2
6,1
7,0
8,1
9,2


In [12]:
original_data

,SEPAL_LENGTH_CM,SEPAL_WIDTH_CM,PETAL_LENGTH_CM,PETAL_WIDTH_CM
0,6.2,2.2,4.5,1.5
1,7.2,3.6,6.1,2.5
2,6.8,3.0,5.5,2.1
3,5.4,3.4,1.7,0.2
4,6.0,2.9,4.5,1.5
5,6.0,3.4,4.5,1.6
6,5.8,2.7,5.1,1.9
7,6.7,3.3,5.7,2.5
8,5.0,3.5,1.3,0.3
9,5.0,3.0,1.6,0.2


In [11]:
remote_prediction.show()

----------------------------------------------------------------------------------------------------
|"SEPAL_LENGTH_CM"  |"SEPAL_WIDTH_CM"  |"PETAL_LENGTH_CM"  |"PETAL_WIDTH_CM"  |"output_feature_0"  |
----------------------------------------------------------------------------------------------------
|6.2                |2.2               |4.5                |1.5               |1                   |
|7.2                |3.6               |6.1                |2.5               |2                   |
|6.8                |3.0               |5.5                |2.1               |2                   |
|5.4                |3.4               |1.7                |0.2               |0                   |
|6.0                |2.9               |4.5                |1.5               |1                   |
|6.0                |3.4               |4.5                |1.6               |1                   |
|5.8                |2.7               |5.1                |1.9               |2           

In [ ]:
remote_prediction = mv.run(x_test_df, function_name="predict_proba")

In [ ]:
remote_prediction.show()

In [ ]:
remote_prediction_1 = mv.run(x_test_df, function_name="predict")

In [ ]:
x_test_df.write.save_as_table("Iris_inference_dataset", mode="overwrite")

In [ ]:
remote_prediction_1.show()

In [ ]:
custom_test_model.context.model_refs.keys()

In [ ]:
!python -c "import platform;print(platform.python_version())"

In [ ]:
model_reg.log_model(
       model=custom_test_model,
       model_name="custom_test_model_feature_pre",
       version_name="custom_test_model_v1",
       comment="Testing score function with custom_test_model",
       conda_dependencies=['scikit-learn==1.3.2'],
       metrics={},
       sample_input_data=m_df.drop('Target',axis=1),
       python_version="3.10.13"
)

In [ ]:
from snowflake.ml.modeling.metrics import (confusion_matrix,
                                        accuracy_score,
                                        f1_score, recall_score,
                                        precision_score,
                                        log_loss ,
                                        roc_auc_score ,
                                        roc_curve)

In [ ]:
roc_auc_score(df=self.sf_df,y_true_col_names=self.true_cn, y_score_col_names=self.pred_cn)

In [ ]:
from fosforml import register_model
register_model(
  model_obj=custom_test_model,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="Snowflake_dataset",
  dataset_source="Snowflake",
  name="SklearnMulitClassModel",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

#### Binary Classfication

In [ ]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_breast_cancer() 
feature_names = m_dataset.feature_names
u_features = [feature.replace(" ","_") for feature in feature_names]
m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target


x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

In [ ]:
session

In [ ]:
from fosforml import register_model
register_model(
  model_obj=model,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="Binary_Model",
  dataset_source = "SNOWFLAKE_STAGE",
  name="SklearnBinaryModel",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

#### Regression

In [ ]:
from sklearn.ensemble._forest import RandomForestRegressor
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_diabetes() 
feature_names = m_dataset.feature_names
u_features = [feature.replace(" ","_") for feature in feature_names]

m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target

x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestRegressor()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

In [ ]:
from fosforml import register_model
register_model(
  name="SklearnReyhtyjhgressionModel",
  model_obj=model,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="RegressionModel",
  dataset_source = "SNOWFLAKE_STAGE",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"]
)